In [1]:
# Path & Libraries

import sys
import os
import warnings

sys.path.append(os.path.abspath('dss'))
warnings.filterwarnings('ignore')

from datetime import datetime
import numpy as np
import pandas as pd
import pickle
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import zipfile

import random
from deap import base, creator, tools, algorithms
from sklearn.metrics import mean_squared_error

from dss.package.dss_package import (select_data_ml, load_weather_data, combine_data_dss, load_true_irrigation, multiple_recommendation)  # Custom Package

# Parameters of Interest

file_path = '../data/dss_data_rovere.csv'
current_date = '2023-06-20'
days_predicted = 3

sensor_id = 72
latitude, longitude = 46.245079, 11.164434


# Data Loading & Processing

# 1) CSV File with Rules & Features Definition

rules_excel_path = '../data/dss_rules.xlsx'
sheet_names = ['Rules', 'Crispy Values', 'Fuzzy Values']

rules_df, crispy_df, fuzzy_df = [pd.read_excel(rules_excel_path, sheet_name=sheet) for sheet in sheet_names]
rules_df = rules_df.iloc[:, :].rename(columns=lambda x: x.lower().replace(' ', '_'))
fuzzy_df = fuzzy_df.iloc[:, :].rename(columns=lambda x: x.lower().replace(' ', '_'))

# 2) Past Tensiometer & Weather Data (Last 3 Days)
data_ml = select_data_ml(file_path, sensor_id, current_date, days_predicted)

# 3) Weather Forecast Data from OpenMeteo API (Next 3 Days)
weather_set = load_weather_data(latitude, longitude, current_date, days_predicted)

# 4) Final Dataset for DSS
data_dss = combine_data_dss(data_ml, weather_set)  # it combines the data from the previous two steps

# 5) ML Model for Predicting Tensiometer Value for the Next Day (it could be used iteratively)

zip_path = 'models_ml.zip'
model_name = 'global_xgb.pkl'
with zipfile.ZipFile(zip_path, 'r') as zipf:
    with zipf.open(model_name) as file:
        xgb_model = pickle.load(file)

# 6) Real Irrigation Data for the Next 3 Days (OPTIONAL)
true_irrigation = load_true_irrigation(file_path, sensor_id, current_date, days_predicted)

# 7) Validation Data to optimize the DSS (OPTIONAL)

# val_csv_path = '../data/boost_val_data.csv'
# data_validation = pd.read_csv(val_csv_path)


# DSS Architecture

# Fuzzy Input Variables (Universe & Membership Functions)

input_variables = rules_df.columns[:-1].tolist()
crispy_input = crispy_df.iloc[:, 0].dropna().values
crispy_output = crispy_df.iloc[:, 1].dropna().values

fuzzy_values = fuzzy_df.values
fuzzy_min = fuzzy_values[0]
fuzzy_max = fuzzy_values[1]
num_classes = fuzzy_values[2].astype(int)

feature_dict = {}
universe_dict = {}

for var, min_val, max_val, num_class in zip(input_variables, fuzzy_min[:-1], fuzzy_max[:-1], num_classes[:-1]):
    feature_dict[var] = {}
    fraction_range = (max_val - min_val) / (num_class - 1)
    universe = np.arange(min_val - 2 * fraction_range, max_val + 2 * fraction_range + 1, 1)
    universe_dict[var] = universe
    for i, term_name in enumerate(crispy_input):
        if i == 0:
            feature_dict[var] = ctrl.Antecedent(universe, var)
            term_range = [min_val - 2 * fraction_range, min_val - 2 * fraction_range, min_val, min_val + fraction_range]
            feature_dict[var][term_name] = fuzz.trapmf(feature_dict[var].universe, term_range)
        elif i == num_class - 1:
            term_range = [max_val - fraction_range, max_val, max_val + 2 * fraction_range, max_val + 2 * fraction_range]
            feature_dict[var][term_name] = fuzz.trapmf(feature_dict[var].universe, term_range)
        else:
            term_range = [min_val + (i - 1) * fraction_range, min_val + i * fraction_range,
                          min_val + (i + 1) * fraction_range]
            feature_dict[var][term_name] = fuzz.trimf(feature_dict[var].universe, term_range)

# Fuzzy Output Variable

output_variable = rules_df.columns[-1]

var = output_variable
min_val = fuzzy_min[-1]
max_val = fuzzy_max[-1]
num_class = num_classes[-1]

feature_dict[var] = {}
fraction_range = (max_val - min_val) / (num_class - 1)
universe = np.arange(min_val - 2 * fraction_range, max_val + 2 * fraction_range + 1, 1)
universe_dict[var] = universe
for i, term_name in enumerate(crispy_output):
    if i == 0:
        feature_dict[var] = ctrl.Consequent(universe, var)
        term_range = [min_val - 2 * fraction_range, min_val - 2 * fraction_range, min_val, min_val + fraction_range]
        feature_dict[var][term_name] = fuzz.trapmf(feature_dict[var].universe, term_range)
    elif i == num_class - 1:
        term_range = [max_val - fraction_range, max_val, max_val + 2 * fraction_range, max_val + 2 * fraction_range]
        feature_dict[var][term_name] = fuzz.trapmf(feature_dict[var].universe, term_range)
    else:
        term_range = [min_val + (i - 1) * fraction_range, min_val + i * fraction_range,
                      min_val + (i + 1) * fraction_range]
        feature_dict[var][term_name] = fuzz.trimf(feature_dict[var].universe, term_range)

# Fuzzy Rules

rule_dict = {}

n_rules = rules_df.shape[0]
n_features = rules_df.shape[1]

for i in range(n_rules):
    term_input = None
    term_output = None

    for j in range(n_features):

        if j == 0:
            term_input = feature_dict[rules_df.columns.tolist()[j]][rules_df.iloc[i, j]]

        elif j == n_features - 1:
            term_output = feature_dict[rules_df.columns.tolist()[j]][rules_df.iloc[i, j]]

        elif not pd.isna(rules_df.iloc[i, j]):
            term_input = term_input & feature_dict[rules_df.columns.tolist()[j]][rules_df.iloc[i, j]]

        else:
            continue

    rule_dict[i] = ctrl.Rule(antecedent=term_input, consequent=term_output)

# No Defuzzification Needed

# Control System Creation & Simulation & Saving

rule_vector = []
for i in range(len(rule_dict)):
    rule_vector.append(rule_dict[i])

dss_ctrl = ctrl.ControlSystem(rule_vector)
dss = ctrl.ControlSystemSimulation(dss_ctrl, flush_after_run=100 * 100 + 1)

In [2]:
np.random.seed(42)
random.seed(42)

val_data_path = '../data/val_data.csv'
val_data = pd.read_csv(val_data_path)

decision_mapping = {
    'Not Recommended': 0.0,
    'Half Turn': 1.0,
    'Single Turn': 2.0,
    'Double Turn': 3.0
}
val_data['Decision'] = val_data['Decision'].map(decision_mapping).astype(float)


def update_fuzzy_system(params):
    lt_low, lt_med, lt_hgh, pt_low, pt_med, pt_hgh, rain_low, rain_med, rain_hgh, tmp_low, tmp_med, tmp_hgh = params
    
    feature_dict['last_avg_tensiometer']['Low'] = fuzz.trapmf(feature_dict['last_avg_tensiometer'].universe, [-100, -100, lt_low, lt_med])
    feature_dict['last_avg_tensiometer']['Medium'] = fuzz.trimf(feature_dict['last_avg_tensiometer'].universe, [lt_low, lt_med, lt_hgh])
    feature_dict['last_avg_tensiometer']['High'] = fuzz.trapmf(feature_dict['last_avg_tensiometer'].universe, [lt_med, lt_hgh, 800, 800])
    
    feature_dict['predicted_avg_tensiometer']['Low'] = fuzz.trapmf(feature_dict['predicted_avg_tensiometer'].universe, [-100, -100, pt_low, pt_med])
    feature_dict['predicted_avg_tensiometer']['Medium'] = fuzz.trimf(feature_dict['predicted_avg_tensiometer'].universe, [pt_low, pt_med, pt_hgh])
    feature_dict['predicted_avg_tensiometer']['High'] = fuzz.trapmf(feature_dict['predicted_avg_tensiometer'].universe, [pt_med, pt_hgh, 800, 800])
    
    feature_dict['predicted_rain_amount']['Low'] = fuzz.trapmf(feature_dict['predicted_rain_amount'].universe, [-20, -20, rain_low, rain_med])
    feature_dict['predicted_rain_amount']['Medium'] = fuzz.trimf(feature_dict['predicted_rain_amount'].universe, [rain_low, rain_med, rain_hgh])
    feature_dict['predicted_rain_amount']['High'] = fuzz.trapmf(feature_dict['predicted_rain_amount'].universe, [rain_med, rain_hgh, 40, 40])
    
    feature_dict['predicted_max_temperature']['Low'] = fuzz.trapmf(feature_dict['predicted_max_temperature'].universe, [-5, -5, tmp_low, tmp_med])
    feature_dict['predicted_max_temperature']['Medium'] = fuzz.trimf(feature_dict['predicted_max_temperature'].universe, [tmp_low, tmp_med, tmp_hgh])
    feature_dict['predicted_max_temperature']['High'] = fuzz.trapmf(feature_dict['predicted_max_temperature'].universe, [tmp_med, tmp_hgh, 55, 55])


init_params = [200, 350, 500, 200, 350, 500, 0, 10, 20, 15, 25, 35]
update_fuzzy_system(init_params)

In [4]:
def objective(params):
    lt_low, lt_med, lt_hgh, pt_low, pt_med, pt_hgh, rain_low, rain_med, rain_hgh, tmp_low, tmp_med, tmp_hgh = params
    
    if not (0 <= lt_low <= lt_med <= lt_hgh <= 799):
        return 1e100, 
    if not (0 <= pt_low <= pt_med <= pt_hgh <= 799):
        return 1e100, 
    if not (0 <= rain_low <= rain_med <= rain_hgh <= 35):
        return 1e100,     
    if not (0 <= tmp_low <= tmp_med <= tmp_hgh <= 50):
        return 1e100,    
    
    update_fuzzy_system(params)
    predictions = []    
    
    for i in range(len(val_data)):
        dss.input['last_avg_tensiometer'] = val_data['Last Avg Tensiometer'][i]
        dss.input['predicted_avg_tensiometer'] = val_data['Predicted Avg Tensiometer'][i]
        dss.input['predicted_rain_amount'] = val_data['Predicted Rain Amount'][i]
        dss.input['predicted_max_temperature'] = val_data['Predicted Max Temperature'][i]
        dss.compute()
        predictions.append(dss.output['decision'])
        
    return mean_squared_error(val_data['Decision'], predictions)


creator.create("FitnessMin", base.Fitness, weights=(-1.0,))  
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("attr_float", np.random.uniform, 0, 100)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, len(input_variables) * 3)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", objective)
toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=10, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)

population = toolbox.population(n=1000)
algorithms.eaSimple(population, toolbox, cxpb=0.5, mutpb=0.3, ngen=100, verbose=False)

best_individual = tools.selBest(population, 1)[0]
print("Best parameters:", best_individual)

Best parameters: [539.5486481143108, 962.8124491074157, 420.97371187459197, -970.3471889185763, -98.32281927463569, 73.39148997083157, 77.59259761596334, -84.64912113918925, -122.86354971989383, -1124.489051513247, -674.1805289055046, 646.7209510976293]


In [9]:
predictions

NameError: name 'predictions' is not defined